In [1]:
import math

def calc_ndcg(relevance_list):
    dcg = 0
    for i, relevance in enumerate(relevance_list):
        dcg += (relevance) / math.log2(i + 1 + 1)
    idcg = 0
    ideal_rank = sorted(relevance_list, reverse=True)
    for i, relevance in enumerate(ideal_rank):
        idcg += (relevance) / math.log2(i + 1 + 1)
    if idcg > 0:
        ndcg = float(dcg) / idcg
    else:
        ndcg = 0
    return ndcg

def calc_mrr(y_pred):
    rr = [1/idx for idx, y in enumerate(y_pred, start=1) if y >= 3]
    
    if not rr:
        return 0.0
    
    return sum(rr) / len(rr)

def calc_recall_at_k(y_pred, k):
    relevant = len([y for y in y_true if y >= 3])
    if y_pred is None or not relevant:
        return None
    predicted = len([y for y in y_pred[:k] if y >= 3])

    return float(predicted) / float(relevant)

def calc_precision_at_k(y_pred, k):
    predicted = len([y for y in y_pred[:k] if y >= 3])

    return float(predicted) / float(k)

def calc_hit_at_k(y_pred, k):
    predicted = len([y for y in y_pred[:k] if y >= 3])
    
    if predicted > 0:
        return 1
    else:
        return 0 

In [2]:
import pandas as pd
import random
import torch

from tqdm import tqdm

tqdm.pandas()

In [3]:
category = 'Games'
signal = 'like'

In [85]:
predictions = pd.read_csv(f'/sise/bshapira-group/lilachzi/models/LlamaRec/experiments/new2/ciao{category}_{signal}/predictions.csv')
predictions['y'] = predictions['y'].apply(eval)
predictions['rank'] = predictions['rank'].apply(eval)
predictions['scores'] = predictions['scores'].apply(eval)

In [86]:
{k:v for k,v in predictions.iloc[:, 2:].mean().items()}

/tmp/ipykernel_32433/1918291347.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  {k:v for k,v in predictions.iloc[:, 2:].mean().items()}


{'nDCG@5': 0.8538683177606385,
 'MRR': 0.699793705289158,
 'Recall@1': 0.5179661058488807,
 'Recall@3': 0.8858619379272233,
 'Recall@5': 0.9538351012524495,
 'Precision@1': 0.6634772373466986,
 'Precision@3': 0.42729493361683807,
 'Precision@5': 0.301135284960322,
 'Hit@1': 0.6634772373466986,
 'Hit@3': 0.9792687094202073,
 'Hit@5': 0.9999620306033337}

In [18]:
test_set = pd.read_csv('../csvs/test_set.csv', converters={'y_true': eval})
test_set = test_set[test_set['category'] == 'Beauty']
test_set

,category,product_id,voter_id,y_true
124,Beauty,86969,6860317,"{187: 3, 169291: 0, 175349: 0, 206266: 0}"
125,Beauty,86969,5781064,"{187: 3, 169291: 0, 175349: 0, 206266: 0}"
126,Beauty,86969,5891,"{169291: 4, 187: 3, 175349: 0, 206266: 0}"
127,Beauty,86969,6772013,"{187: 3, 169291: 0, 175349: 0, 206266: 0}"
128,Beauty,86969,6023030,"{187: 4, 206266: 4, 169291: 0, 175349: 0}"
...,...,...,...,...
392726,Beauty,48892,5001138,"{303517: 3, 121404: 0, 238641: 0, 296833: 0}"
392765,Beauty,25723,5016866,"{303617: 3, 63209: 0, 99720: 0, 113796: 0, 116..."
392887,Beauty,6567,9147,"{304051: 4, 100608: 0, 111481: 0, 208181: 0}"
392946,Beauty,96867,5011839,"{32812: 3.5, 42223: 3.5, 49297: 3.5, 108139: 3..."


In [19]:
test_set['len'] = test_set['y_true'].apply(len)

In [23]:
test_set[(test_set['len'] > 20) & (test_set['len'] <= 50)]

,category,product_id,voter_id,y_true,len
1613,Beauty,19812,5326499,"{1274: 3, 24747: 0, 28306: 0, 51280: 0, 53543:...",33
1614,Beauty,19812,5584168,"{166180: 4, 186301: 4, 290712: 4, 1274: 3, 210...",33
1615,Beauty,19812,5013640,"{106397: 4, 181997: 4, 1274: 3, 24747: 0, 2830...",33
1616,Beauty,19812,5076524,"{88377: 4, 102326: 4, 106397: 4, 107284: 4, 18...",33
1617,Beauty,19812,5285637,"{1274: 4, 24747: 0, 28306: 0, 51280: 0, 53543:...",33
...,...,...,...,...,...
391280,Beauty,73855,23249,"{16004: 3.5, 17159: 3.5, 47214: 3.5, 51070: 3....",32
391286,Beauty,77714,10438,"{3283: 3.5, 19681: 3.5, 31946: 3.5, 81880: 3.5...",24
392390,Beauty,45706,15714,"{302238: 4, 70530: 0, 85761: 0, 126396: 0, 155...",23
392946,Beauty,96867,5011839,"{32812: 3.5, 42223: 3.5, 49297: 3.5, 108139: 3...",23


In [153]:
def calc_user_votes(y_true):
    return len([i for i in y_true.values() if i not in [0, 3.5]])

test_set['y_true'].apply(calc_user_votes).describe()

count    26337.000000
mean         4.374150
std         28.250694
min          0.000000
25%          1.000000
50%          1.000000
75%          2.000000
max        513.000000
Name: y_true, dtype: float64

In [154]:
test_set['votes_count'] = test_set['y_true'].apply(calc_user_votes)

In [158]:
test_set[test_set['votes_count'] == 1].iloc[:, 7:-1].mean()

nDCG@5         0.798777
MRR            0.752233
Recall@1       0.202614
Recall@3       0.335436
Recall@5       0.362842
Precision@1    0.607841
Precision@3    0.335436
Precision@5    0.217705
Hit@1          0.607841
Hit@3          0.960065
Hit@5          0.999948
dtype: float64

In [159]:
test_set[test_set['votes_count'] != 1].iloc[:, 7:-1].mean()

nDCG@5         0.858849
MRR            0.496495
Recall@1       0.247764
Recall@3       0.645892
Recall@5       0.848752
Precision@1    0.743292
Precision@3    0.645892
Precision@5    0.509251
Hit@1          0.743292
Hit@3          0.981274
Hit@5          1.000000
dtype: float64

In [137]:
test_set['y'] = predictions['y'].tolist()
test_set['rank'] = predictions['rank'].tolist()

In [119]:
predictions.iloc[3]

y                                                   [4, 4, 3, 0]
scores         [0.88671875, 0.7421875, 0.50390625, 0.41601562...
rank           [8, 4, 0, 1, 11, 2, 3, 16, 17, 13, 5, 6, 15, 1...
nDCG@5                                                       1.0
MRR                                                     0.611111
Recall@1                                                0.333333
Recall@3                                                     1.0
Recall@5                                                     1.0
Precision@1                                                  1.0
Precision@3                                                  1.0
Precision@5                                                  0.6
Hit@1                                                          1
Hit@3                                                          1
Hit@5                                                          1
Name: 3, dtype: object

In [127]:
y_true = list(test_set.iloc[3]['y_true'].values())
y = predictions.iloc[3]['y']
rank = predictions.iloc[3]['rank']
# rank = [r for r in rank if r < len(y)]
rank

[8, 4, 0, 1, 11, 2, 3, 16, 17, 13, 5, 6, 15, 19, 18, 12, 14, 7, 9, 10]

In [125]:
y_true

[4, 4, 3, 0]

In [140]:
def get_ranked_list(rank, y_true):
    ranked_list = []
    for r in rank:
        if r < len(y_true):
            ranked_list.append(y_true[r])
        else:
            ranked_list.append(-1)
            
    return ranked_list

test_set['ranked_list'] = test_set.progress_apply(
    lambda r: get_ranked_list(r['rank'], list(r['y_true'].values())), axis=1
)

100%|██████████| 26337/26337 [00:00<00:00, 32595.95it/s]


In [141]:
test_set

,category,product_id,voter_id,y_true,y,rank,ranked_list
2103,Internet,106745,5242671,"{1370: 3, 113671: 0, 154395: 0, 265008: 0}","[3, 0, 0, 0]","[0, 2, 1, 3, 4, 5, 8, 6, 19, 15, 18, 11, 12, 1...","[3, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2104,Internet,106745,5203171,"{113671: 4, 1370: 3, 154395: 0, 265008: 0}","[4, 0, 0, 3]","[0, 2, 3, 1, 4, 14, 8, 11, 5, 6, 12, 18, 13, 1...","[4, 0, 0, 3, -1, -1, -1, -1, -1, -1, -1, -1, -..."
2105,Internet,106745,5498542,"{1370: 3, 113671: 0, 154395: 0, 265008: 0}","[3, 0, 0, 0]","[0, 2, 1, 4, 3, 16, 5, 8, 15, 12, 11, 6, 14, 1...","[3, 0, 0, -1, 0, -1, -1, -1, -1, -1, -1, -1, -..."
2106,Internet,106745,9444,"{113671: 4, 154395: 4, 1370: 3, 265008: 0}","[4, 4, 3, 0]","[8, 4, 0, 1, 11, 2, 3, 16, 17, 13, 5, 6, 15, 1...","[-1, -1, 4, 4, -1, 3, 0, -1, -1, -1, -1, -1, -..."
2107,Internet,106745,5373705,"{1370: 3, 113671: 0, 154395: 0, 265008: 0}","[3, 0, 0, 0]","[4, 8, 0, 17, 2, 3, 16, 13, 1, 11, 18, 6, 15, ...","[-1, -1, 3, -1, 0, 0, -1, -1, 0, -1, -1, -1, -..."
...,...,...,...,...,...,...,...
392854,Internet,107002,5002576,"{3733: 3.5, 23237: 3.5, 100725: 3.5, 122813: 3...","[3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, ...","[0, 1, 3, 2, 4, 5, 6, 7, 8, 12, 15, 14, 13, 18...","[3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, ..."
392867,Internet,51225,5011839,"{303943: 4, 110003: 0, 120510: 0, 149249: 0, 1...","[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 4, 2, 1, 3, 5, 9, 6, 11, 8, 16, 14, 7, 15,...","[4, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 0, -1, ..."
392871,Internet,33471,15714,"{61682: 3.5, 109248: 3.5, 111861: 3.5, 152572:...","[3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, ...","[2, 8, 4, 0, 17, 3, 16, 11, 12, 18, 1, 13, 6, ...","[3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, ..."
392878,Internet,14177,13861,"{303972: 4, 149155: 0, 238292: 0, 279891: 0, 2...","[4, 0, 0, 0, 0, 0]","[0, 1, 2, 18, 3, 4, 19, 15, 14, 11, 6, 5, 17, ...","[4, 0, 0, -1, 0, 0, -1, -1, -1, -1, -1, 0, -1,..."


In [143]:
test_set['nDCG@5'] = test_set.apply(lambda x: calc_ndcg(x['ranked_list'][:5]), axis=1)
test_set['MRR'] = test_set.apply(lambda x: calc_mrr(x['ranked_list']), axis=1)
test_set['Recall@1'] = test_set.apply(lambda x: calc_recall_at_k(x['ranked_list'], k=1), axis=1)
test_set['Recall@3'] = test_set.apply(lambda x: calc_recall_at_k(x['ranked_list'], k=3), axis=1)
test_set['Recall@5'] = test_set.apply(lambda x: calc_recall_at_k(x['ranked_list'], k=5), axis=1)
test_set['Precision@1'] = test_set.apply(lambda x: calc_precision_at_k(x['ranked_list'], k=1), axis=1)
test_set['Precision@3'] = test_set.apply(lambda x: calc_precision_at_k(x['ranked_list'], k=3), axis=1)
test_set['Precision@5'] = test_set.apply(lambda x: calc_precision_at_k(x['ranked_list'], k=5), axis=1)
test_set['Hit@1'] = test_set.apply(lambda x: calc_hit_at_k(x['ranked_list'], k=1), axis=1)
test_set['Hit@3'] = test_set.apply(lambda x: calc_hit_at_k(x['ranked_list'], k=3), axis=1)
test_set['Hit@5'] = test_set.apply(lambda x: calc_hit_at_k(x['ranked_list'], k=5), axis=1)

In [146]:
test_set.iloc[:, 7:].mean()

nDCG@5         0.815099
MRR            0.682747
Recall@1       0.214881
Recall@3       0.419790
Recall@5       0.494868
Precision@1    0.644644
Precision@3    0.419790
Precision@5    0.296921
Hit@1          0.644644
Hit@3          0.965828
Hit@5          0.999962
dtype: float64

In [3]:
eval_df = pd.read_csv('/sise/bshapira-group/lilachzi/models/LlamaRec/experiments/random_full/ciaoGames_like_test4/eval_predictions_2024_04_08_06_17.csv')

In [9]:
eval_df = eval_df.dropna()

In [20]:
eval_df['len'] = eval_df['y'].apply(lambda l: len(eval(l)))

/tmp/ipykernel_1875919/415502483.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eval_df['len'] = eval_df['y'].apply(lambda l: len(eval(l)))


In [21]:
eval_df[eval_df['len'] > 20].iloc[:, 2:-1].mean()

nDCG@5         0.190441
MRR            0.107488
Recall@1       0.026518
Recall@3       0.077184
Recall@5       0.127479
Precision@1    0.081089
Precision@3    0.081878
Precision@5    0.081740
Hit@1          0.081089
Hit@3          0.156851
Hit@5          0.223439
dtype: float64

In [22]:
eval_df[eval_df['len'] <= 20].iloc[:, 2:-1].mean()

nDCG@5         0.478796
MRR            0.366436
Recall@1       0.150337
Recall@3       0.440919
Recall@5       0.692633
Precision@1    0.204042
Precision@3    0.200929
Precision@5    0.190692
Hit@1          0.204042
Hit@3          0.507609
Hit@5          0.738707
dtype: float64

In [23]:
len(eval_df[eval_df['len'] > 20]), len(eval_df[eval_df['len'] <= 20])

(3379, 12419)

In [39]:
# eval_df = pd.read_csv('/sise/bshapira-group/lilachzi/models/LlamaRec/experiments/random_full/ciaoGames_like_test4/eval_predictions_2024_04_08_06_17.csv')
eval_df = pd.read_csv('/sise/bshapira-group/lilachzi/models/LlamaRec/experiments/random_full/ciaoGames_like_test5/eval_predictions_2024_04_08_15_21.csv')

In [40]:
eval_df = eval_df.dropna()

In [41]:
eval_df['len'] = eval_df['y'].apply(lambda l: len(eval(l)))

In [42]:
eval_df[eval_df['len'] > 20].iloc[:, 2:-1].mean()

nDCG@5         0.191395
MRR            0.106994
Recall@1       0.027948
Recall@3       0.073985
Recall@5       0.127596
Precision@1    0.086120
Precision@3    0.080201
Precision@5    0.081563
Hit@1          0.086120
Hit@3          0.150932
Hit@5          0.221959
dtype: float64

In [43]:
eval_df[eval_df['len'] <= 20].iloc[:, 2:-1].mean()

nDCG@5         0.480340
MRR            0.365606
Recall@1       0.145120
Recall@3       0.445426
Recall@5       0.698128
Precision@1    0.198567
Precision@3    0.202405
Precision@5    0.192318
Hit@1          0.198567
Hit@3          0.512119
Hit@5          0.744102
dtype: float64

In [44]:
len(eval_df[eval_df['len'] > 20]), len(eval_df[eval_df['len'] <= 20])

(3379, 12419)

In [45]:
eval_df.iloc[:, 2:-1].mean()

nDCG@5         0.418538
MRR            0.310292
Recall@1       0.120059
Recall@3       0.365979
Recall@5       0.576098
Precision@1    0.174516
Precision@3    0.176267
Precision@5    0.168629
Hit@1          0.174516
Hit@3          0.434865
Hit@5          0.632422
dtype: float64

In [54]:
df = pd.read_csv('/sise/bshapira-group/lilachzi/models/LlamaRec/experiments/random_full/ciaoGames_like_test4/predictions.csv')

In [57]:
# df = df.dropna()
df

,y,rank,nDCG@5,MRR,Recall@1,Recall@3,Recall@5,Precision@1,Precision@3,Precision@5,Hit@1,Hit@3,Hit@5
0,"[0, 0, 4, 0, 4, 0]","tensor([ 0, 1, 3, 4, 2, 5, -1, -1, -1, -1...",0.543771,0.266667,0.0,0.5,1.0,0.0,0.333333,0.4,0,1,1
1,"[4, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0]","tensor([ 8, 9, 0, 10, 4, 1, 3, 2, 5, 12...",1.000000,0.571429,0.5,0.5,0.5,1.0,0.333333,0.2,1,1,1
2,"[0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","tensor([16, 36, 9, 48, 49, 29, 8, 28, 40, 50...",0.430677,0.250000,0.0,0.0,1.0,0.0,0.000000,0.2,0,0,1
3,"[0, 0, 0, 0, 0, 3, 0]","tensor([ 0, 4, 1, 3, 5, 6, 2, -1, -1, -1...",0.000000,0.166667,0.0,0.0,0.0,0.0,0.000000,0.0,0,0,0
4,"[0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","tensor([ 8, 16, 9, 0, 1, 2, 3, 5, 12, 10...",0.386853,0.200000,0.0,0.0,1.0,0.0,0.000000,0.2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15760,"[0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0]","tensor([-1, 9, 8, 0, 1, 2, -1, 10, 5, 4...",0.000000,0.166667,0.0,0.0,0.0,0.0,0.000000,0.0,0,0,0
15761,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4]","tensor([ 0, 8, 9, -1, 1, 2, 12, 3, 4, 5...",0.000000,0.066667,0.0,0.0,0.0,0.0,0.000000,0.0,0,0,0
15762,"[3.5, 1.0, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, 3.5, ...","tensor([ 8, 0, 1, 4, 3, 5, 9, 2, 6, 7...",0.934756,0.251988,0.1,0.2,0.4,1.0,0.666667,0.8,1,1,1
15763,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","tensor([ 0, 1, 3, 2, 4, 5, 20, 21, 23, 24...",0.000000,0.052632,0.0,0.0,0.0,0.0,0.000000,0.0,0,0,0
